In [242]:
import pandas as pd
import random
import googlemaps
from datetime import datetime

import tsp #solves the travelling salesman problem

class GoogleMapsClient():
    def __init__(self):
        
        self.gmaps = googlemaps.Client(key=os.environ['mapsAPI'])
        self.poi_latlon = None
        
    def get_place_ids_from_POIs(self,pois):
        #accept list of pois
        
        candidates = []
        
        for i in range(len(pois)):
            location_bias_string = f"point:{pois[i]['geoCode']['latitude']},{pois[i]['geoCode']['longitude']}"
            candidate = self.gmaps.find_place(input = pois[i]['name'], input_type='textquery',fields=['place_id'],location_bias=location_bias_string)
            
            candidates.append(candidate['candidates'][0]['place_id']) #extracts the first poi

        return candidates
    
    def get_mean_location(self,pois):
        #uses pois returned by amadeus
        poi_latlon = [pois[i]['geoCode'] for i in range(len(pois))]
        
        return pd.DataFrame.from_dict(poi_latlon).mean().to_dict()
        

    def generate_itinerary(self, pois, hotel_lat_lon = None):
        #assume hotel is dictionary of  in the style of {'latitude': 50.085133, 'longitude': 14.424776}
        #if not provided, the hotel is assumed to be at the mean location of the pois.
        #pois are list of pois generated by amadeus
        #result is a dictionary, that has:
        #    result['path'] is the path to be taken, indexed by pois
        #    result['travel_time'] is the matrix of travel times from a poi (row) to another poi (col). Note, indexes are shifted up by 1 due to hotel being added as index 0.
        #    result['dist_matrix'] is the distances matrix returned by google.
        
        if hotel_lat_lon is None:
            hotel_lat_lon = self.get_mean_location(pois)
            
        #convert pois to lat long
        poi_latlon = self.poi_latlon = [pois[i]['geoCode'] for i in range(len(pois))]
        #saved for future reference
        
        
        #collate
        full_latlon = [hotel_lat_lon] + poi_latlon
        num_of_points = len(full_latlon)
        
        #generate distance matrix from google
        dist_matrix = self.gmaps.distance_matrix(full_latlon,full_latlon,mode='transit')
        
        #generate summary of travel times (each number is the number of seconds of travel)
        travel_time = [[dist_matrix['rows'][o]['elements'][d]['duration']['value'] for d in range(num_of_points)] for o in range(num_of_points)]
        
        
        #convert to dictionary
        r = range(len(travel_time))
        cost = {(i,j): travel_time[i][j] for i in r for j in r}
        
        #solve travelling salesman problem
        (totaltime, path) = tsp.tsp(r, cost)
        
        #correct index, because hotel was added at point 0
        poi_path = [loc-1 for loc in path[1:]]    
        
        result = dict()
        result['path'] = poi_path
        result['dist_matrix'] = dist_matrix
        result['travel_time'] = travel_time
        
        return result
                                              
        
    def get_transit_directions(origin, destination, location_bias, mode='transit'):
        #assume origin is either in a amadeus style or is a text entry for google maps search
        
        if type(origin) is str:
            
        
        

In [247]:
type('Ritz') is str

True

In [184]:
import os
os.chdir('../data-import/')

from AmadeusClient import AmadeusClient

am = AmadeusClient()

os.chdir('../planner/')

In [185]:
am

In [186]:
pois = am.get_poi(lat = 50.089848, lon = 14.432865)
pois = random.sample(pois, k = 10)
pois

[{'type': 'location',
  'subType': 'POINT_OF_INTEREST',
  'geoCode': {'latitude': 50.092182, 'longitude': 14.449174},
  'name': 'Charleston',
  'category': 'RESTAURANT',
  'tags': ['restaurant'],
  'city': ''},
 {'type': 'location',
  'subType': 'POINT_OF_INTEREST',
  'geoCode': {'latitude': 50.08469, 'longitude': 14.423233},
  'name': 'U Provaznice',
  'category': 'RESTAURANT',
  'tags': ['restaurant'],
  'city': ''},
 {'type': 'location',
  'subType': 'POINT_OF_INTEREST',
  'geoCode': {'latitude': 50.086975, 'longitude': 14.425542},
  'name': 'Grand Café Orient',
  'category': 'RESTAURANT',
  'tags': ['sightseeing', 'restaurant', 'bar', 'nightlife'],
  'city': ''},
 {'type': 'location',
  'subType': 'POINT_OF_INTEREST',
  'geoCode': {'latitude': 50.088314, 'longitude': 14.416279},
  'name': 'Mistral Café',
  'category': 'RESTAURANT',
  'tags': ['sightseeing', 'restaurant'],
  'city': ''},
 {'type': 'location',
  'subType': 'POINT_OF_INTEREST',
  'geoCode': {'latitude': 50.098404, 'lo

In [243]:
client = GoogleMapsClient()

In [225]:
placeids = client.get_place_ids_from_POIs(pois)

In [199]:
placeids

['ChIJlxSKQqOUC0cROf8ruWvkSOg',
 'ChIJ4ZUSFeyUC0cRV7WrymAISVw',
 'ChIJ-0VKo-uUC0cRK2ObH735hTs',
 'ChIJ9XdFnuiUC0cR78EyLOsi128',
 'ChIJ-wd_SsSUC0cRpJbY3tlQRHs',
 'ChIJ4zz7t-eUC0cRYoyBVOzwQ-o',
 'ChIJf3Fg0OeUC0cRD_AhFLhTWxQ',
 'ChIJMfepuO6UC0cRyCS058Lx7GI',
 'ChIJ-eBkvOmUC0cRCAA9WW_C--0',
 'ChIJL2WboO2UC0cRlw47PEdAPVQ']

In [244]:
hotel=client.get_mean_location(pois)

In [245]:
len(pois[:-1])

9

In [246]:
client.generate_itinerary(pois = pois[:-1], hotel_lat_lon= hotel)

{'dist_matrix': {'destination_addresses': ['Týnská 21, 110 00 Staré Město, Czechia',
   'Křižíkova 208/55, 186 00 Karlín, Czechia',
   'Provaznická 388/2, 110 00 Praha-Staré Město, Czechia',
   'Ovocný trh 19, 110 00 Praha 1-Staré Město, Czechia',
   'Valentinská 56/11, 110 00 Praha-Staré Město, Czechia',
   'Letohradská 648/34, 170 00 Praha 7-Holešovice, Czechia',
   'Alšovo nábř. 12, 110 00 Praha 1-Staré Město, Czechia',
   '17. listopadu 2, 110 00 Praha 1-Staré Město, Czechia',
   'Michalská 433/14, 110 00 Praha-Staré Město, Czechia',
   'V Kolkovně 921/3, 110 00 Praha-Staré Město, Czechia'],
  'origin_addresses': ['Týnská 21, 110 00 Staré Město, Czechia',
   'Křižíkova 208/55, 186 00 Karlín, Czechia',
   'Provaznická 388/2, 110 00 Praha-Staré Město, Czechia',
   'Ovocný trh 19, 110 00 Praha 1-Staré Město, Czechia',
   'Valentinská 56/11, 110 00 Praha-Staré Město, Czechia',
   'Letohradská 648/34, 170 00 Praha 7-Holešovice, Czechia',
   'Alšovo nábř. 12, 110 00 Praha 1-Staré Město, 

In [117]:
gmaps.directions(placeids[0], placeids[1],mode='transit',departure_time=now)

HTTPError: HTTP Error: 400

In [128]:
gmaps.distance_matrix(placeids[0], placeids[1])

HTTPError: HTTP Error: 400

In [67]:
f"point:{pois[0]['geoCode']['latitude']},{pois[0]['geoCode']['longitude']}"

'point:50.085133,14.424776'

In [68]:
candidates = gmaps.find_place(input = pois[0]['name'], input_type='textquery',fields=['place_id'],location_bias=f"point:{pois[0]['geoCode']['latitude']},{pois[0]['geoCode']['longitude']}")

In [69]:
placeid = candidates['candidates'][0]['place_id']

In [70]:
placeid

'ChIJqSCReOuUC0cREEclXxCXQVM'

In [100]:


gmaps = googlemaps.Client(key=os.environ['mapsAPI'])

# Geocoding an address
geocode_result = gmaps.geocode('wework old street')

# Look up an address with reverse geocoding
reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))

# Request directions via public transit
now = datetime.now()
directions_result = gmaps.directions("Sydney Town Hall",
                                     "Parramatta, NSW",
                                     mode="transit",
                                     departure_time=now)

In [ ]:
gmaps.geocode('')

In [34]:
geocode_result

[{'address_components': [{'long_name': '41',
    'short_name': '41',
    'types': ['street_number']},
   {'long_name': 'Corsham Street',
    'short_name': 'Corsham St',
    'types': ['route']},
   {'long_name': 'Old Street',
    'short_name': 'Old Street',
    'types': ['neighborhood', 'political']},
   {'long_name': 'London', 'short_name': 'London', 'types': ['postal_town']},
   {'long_name': 'Greater London',
    'short_name': 'Greater London',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'England',
    'short_name': 'England',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United Kingdom',
    'short_name': 'GB',
    'types': ['country', 'political']},
   {'long_name': 'N1 6DR', 'short_name': 'N1 6DR', 'types': ['postal_code']}],
  'formatted_address': '41 Corsham St, Old Street, London N1 6DR, UK',
  'geometry': {'location': {'lat': 51.5272594, 'lng': -0.0870205},
   'location_type': 'ROOFTOP',
   'viewport': {'northeas

In [35]:
geocode_result[0]['place_id']

'ChIJgy8CyaUcdkgRkAWC-kAcUW8'

In [37]:
deets = gmaps.place(geocode_result[0]['place_id'])

In [38]:
deets

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '41',
    'short_name': '41',
    'types': ['street_number']},
   {'long_name': 'Corsham Street',
    'short_name': 'Corsham St',
    'types': ['route']},
   {'long_name': 'Old Street',
    'short_name': 'Old Street',
    'types': ['neighborhood', 'political']},
   {'long_name': 'London', 'short_name': 'London', 'types': ['postal_town']},
   {'long_name': 'Greater London',
    'short_name': 'Greater London',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'England',
    'short_name': 'England',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United Kingdom',
    'short_name': 'GB',
    'types': ['country', 'political']},
   {'long_name': 'N1 6DR', 'short_name': 'N1 6DR', 'types': ['postal_code']}],
  'adr_address': '<span class="street-address">41 Corsham St</span>, <span class="locality">London</span> <span class="postal-code">N1 6DR</span>, <span class

In [41]:
deets['result']['photos']

[{'height': 426,
  'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102939174042935545234/photos">WeWork 41 Corsham St</a>'],
  'photo_reference': 'CmRaAAAAtalm9OZgfCq58PEbGlMHji3yfWr6JIj1Q4XwFELDksszDqADz6k_YVvGL65oCwHXqxSf8ncdjhD_h57BvCzWvSS2a6gSyQ_UEabuDAqpAZRE50YPhEeAALRrljrQ6QZ3EhAEkKot3M3hrQO6uJzKM3W2GhTvlKDwPK2gPGz-QFWb5_XUou32Rw',
  'width': 758},
 {'height': 2640,
  'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102939174042935545234/photos">WeWork 41 Corsham St</a>'],
  'photo_reference': 'CmRaAAAA74t1xX68coAzdO8h-nQuQHQuI5mo9g6FVJpITZMwAxALRRyOZXTYNEeAL5K3oTsV7q77V1Ia28GsqAWeBe4E-tzWPN-gH5DLNE33eIhrMZWQsbjWLYR6sjtZ8lWi0qvbEhD2b3z6SBFQSuqJyK2OJGalGhRuKKc9vseUCnsJ_00HJ6oKpJYYFA',
  'width': 3960},
 {'height': 3024,
  'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109014961715875174436/photos">Amar Patel</a>'],
  'photo_reference': 'CmRaAAAAcWi2XFaF05stQ94oL8ssZY-HdcrZYphbXx7__N0VQ3s-om66G0_ML9PdW3zAn_JiAq8C7s8SU6O

In [43]:
deets['result']['photos'][0]['html_attributions']

['<a href="https://maps.google.com/maps/contrib/102939174042935545234/photos">WeWork 41 Corsham St</a>']

In [44]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://maps.google.com/maps/contrib/102939174042935545234/photos")

In [122]:
pois[0]['geoCode']

{'latitude': 50.085133, 'longitude': 14.424776}

In [124]:
gmaps.directions(pois[0]['geoCode'],pois[1]['geoCode'],mode='transit')

[{'bounds': {'northeast': {'lat': 50.09358, 'lng': 14.4553678},
   'southwest': {'lat': 50.08361, 'lng': 14.42388}},
  'copyrights': 'Map data ©2019 Google',
  'fare': {'currency': 'CZK', 'text': 'CZK\xa024.00', 'value': 24},
  'legs': [{'arrival_time': {'text': '8:19pm',
     'time_zone': 'Europe/Prague',
     'value': 1564337997},
    'departure_time': {'text': '8:06pm',
     'time_zone': 'Europe/Prague',
     'value': 1564337199},
    'distance': {'text': '3.0 km', 'value': 3040},
    'duration': {'text': '13 mins', 'value': 798},
    'end_address': 'Křižíkova 386/105, 186 00 Karlín, Czechia',
    'end_location': {'lat': 50.0935618, 'lng': 14.4553186},
    'start_address': 'Swarovski Boutique, Na Příkopě 15, 110 00 Praha-Staré Město, Czechia',
    'start_location': {'lat': 50.0850945, 'lng': 14.4248348},
    'steps': [{'distance': {'text': '0.2 km', 'value': 183},
      'duration': {'text': '3 mins', 'value': 161},
      'end_location': {'lat': 50.08361, 'lng': 14.42398},
      'htm

In [129]:
origins = [pois[i]['geoCode'] for i in range(len(pois))]
destinations = [pois[i]['geoCode'] for i in range(len(pois))]

gmaps.directions(origins[0],origins[1],mode='transit')

[{'bounds': {'northeast': {'lat': 50.09358, 'lng': 14.4553678},
   'southwest': {'lat': 50.08361, 'lng': 14.42388}},
  'copyrights': 'Map data ©2019 Google',
  'fare': {'currency': 'CZK', 'text': 'CZK\xa024.00', 'value': 24},
  'legs': [{'arrival_time': {'text': '8:34pm',
     'time_zone': 'Europe/Prague',
     'value': 1564338897},
    'departure_time': {'text': '8:21pm',
     'time_zone': 'Europe/Prague',
     'value': 1564338099},
    'distance': {'text': '3.0 km', 'value': 3040},
    'duration': {'text': '13 mins', 'value': 798},
    'end_address': 'Křižíkova 386/105, 186 00 Karlín, Czechia',
    'end_location': {'lat': 50.0935618, 'lng': 14.4553186},
    'start_address': 'Swarovski Boutique, Na Příkopě 15, 110 00 Praha-Staré Město, Czechia',
    'start_location': {'lat': 50.0850945, 'lng': 14.4248348},
    'steps': [{'distance': {'text': '0.2 km', 'value': 183},
      'duration': {'text': '3 mins', 'value': 161},
      'end_location': {'lat': 50.08361, 'lng': 14.42398},
      'htm

In [134]:
dist_matrix = gmaps.distance_matrix(origins,destinations,mode='transit')

In [178]:
pd.DataFrame.from_dict(origins).mean()

latitude     50.087013
longitude    14.430674
dtype: float64

In [179]:
origins

[{'latitude': 50.085133, 'longitude': 14.424776},
 {'latitude': 50.09354, 'longitude': 14.455214},
 {'latitude': 50.08562, 'longitude': 14.431791},
 {'latitude': 50.08641, 'longitude': 14.420797},
 {'latitude': 50.09586, 'longitude': 14.427088},
 {'latitude': 50.08884, 'longitude': 14.419975},
 {'latitude': 50.086445, 'longitude': 14.42011},
 {'latitude': 50.083973, 'longitude': 14.414278},
 {'latitude': 50.081203, 'longitude': 14.444147},
 {'latitude': 50.08311, 'longitude': 14.44856}]

In [148]:
ll = [[dist_matrix['rows'][o]['elements'][d]['duration']['value'] for d in range(len(destinations))] for o in range(len(origins))]

In [149]:
ll

[[0, 798, 478, 300, 1094, 416, 326, 683, 1152, 1138],
 [867, 0, 840, 1019, 1349, 1244, 1086, 1276, 1832, 1511],
 [472, 776, 0, 704, 958, 839, 771, 736, 902, 781],
 [313, 970, 672, 0, 955, 210, 65, 538, 1424, 1303],
 [1001, 1132, 813, 995, 0, 773, 976, 773, 1438, 1317],
 [444, 1404, 869, 224, 914, 0, 224, 567, 1378, 1554],
 [346, 1043, 745, 70, 961, 215, 0, 499, 1391, 1438],
 [714, 1358, 767, 565, 1140, 658, 512, 0, 1519, 1398],
 [972, 1377, 743, 1129, 1552, 1175, 1196, 1329, 0, 361],
 [891, 1381, 575, 1129, 1315, 1279, 1285, 1161, 367, 0]]

In [141]:
dist_matrix['rows'][0]['elements'][1]

{'distance': {'text': '3.0 km', 'value': 3040},
 'duration': {'text': '13 mins', 'value': 798},
 'fare': {'currency': 'CZK', 'text': 'CZK\xa024.00', 'value': 24},
 'status': 'OK'}

In [154]:
#algo to find best path through some weights
#assume there is a visited list, where you dont want to go to 
#assume you start at the origin, the next node is simply the next nearest node. 

path = [0]
visited = set([0])
current_location = 0

In [153]:
visited

{0}

In [155]:
ll[current_location]

[0, 798, 478, 300, 1094, 416, 326, 683, 1152, 1138]

In [157]:
min(ll[current_location])

0

In [163]:
import numpy as np
lla = np.array(ll)

In [164]:
lla

array([[   0,  798,  478,  300, 1094,  416,  326,  683, 1152, 1138],
       [ 867,    0,  840, 1019, 1349, 1244, 1086, 1276, 1832, 1511],
       [ 472,  776,    0,  704,  958,  839,  771,  736,  902,  781],
       [ 313,  970,  672,    0,  955,  210,   65,  538, 1424, 1303],
       [1001, 1132,  813,  995,    0,  773,  976,  773, 1438, 1317],
       [ 444, 1404,  869,  224,  914,    0,  224,  567, 1378, 1554],
       [ 346, 1043,  745,   70,  961,  215,    0,  499, 1391, 1438],
       [ 714, 1358,  767,  565, 1140,  658,  512,    0, 1519, 1398],
       [ 972, 1377,  743, 1129, 1552, 1175, 1196, 1329,    0,  361],
       [ 891, 1381,  575, 1129, 1315, 1279, 1285, 1161,  367,    0]])

In [165]:
np.factorial(10)

AttributeError: module 'numpy' has no attribute 'factorial'

In [174]:
import tsp

In [167]:
r = range(len(ll))

In [168]:
r

range(0, 10)

In [169]:
ll

[[0, 798, 478, 300, 1094, 416, 326, 683, 1152, 1138],
 [867, 0, 840, 1019, 1349, 1244, 1086, 1276, 1832, 1511],
 [472, 776, 0, 704, 958, 839, 771, 736, 902, 781],
 [313, 970, 672, 0, 955, 210, 65, 538, 1424, 1303],
 [1001, 1132, 813, 995, 0, 773, 976, 773, 1438, 1317],
 [444, 1404, 869, 224, 914, 0, 224, 567, 1378, 1554],
 [346, 1043, 745, 70, 961, 215, 0, 499, 1391, 1438],
 [714, 1358, 767, 565, 1140, 658, 512, 0, 1519, 1398],
 [972, 1377, 743, 1129, 1552, 1175, 1196, 1329, 0, 361],
 [891, 1381, 575, 1129, 1315, 1279, 1285, 1161, 367, 0]]

In [170]:
ll[2][3]

704

In [171]:
dist = {(i,j): ll[i][j] for i in r for j in r}

In [175]:
print(tsp.tsp(r, dist))

(6222.0, [0, 8, 9, 2, 1, 4, 7, 6, 3, 5])


In [180]:
path = [0, 8, 9, 2, 1, 4, 7, 6, 3, 5]

In [182]:
[loc-1 for loc in path[1:]]

[7, 8, 1, 0, 3, 6, 5, 2, 4]